In [ ]:
%pip install python-dotenv alpaca-py requests pandas

In [ ]:
%reload_ext dotenv
%dotenv

In [ ]:
from alpaca.data.historical.crypto import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame
import pandas as pd
from os import getenv
from datetime import datetime
import pytz
from itertools import product

In [ ]:
# Get supported pairs
import requests
r = requests.get(url='https://api.alpaca.markets/v2/assets?asset_class=crypto', headers={'Apca-Api-Key-Id': getenv('KEY_ID'), 'Apca-Api-Secret-Key': getenv('SECRET_KEY')})
assets = []
for x in r.json():
    assets.append(x["symbol"])
print(len(assets))

In [ ]:
client = CryptoHistoricalDataClient(api_key=getenv('KEY_ID'), secret_key=getenv('SECRET_KEY'))
req = CryptoBarsRequest(symbol_or_symbols=assets, timeframe=TimeFrame.Minute, start=datetime(year=2008, month=1, day=1))
bars = client.get_crypto_bars(request_params=req, feed="us")
df = bars.df

In [ ]:
currencies = df.index.get_level_values(0).unique()
times = df.index.get_level_values(1).unique()
idx = df.index
pairs = list(product(currencies, times))
missing = [x for x in pairs if x not in idx]
missing_currencies = pd.Series([x[0] for x in missing]).unique().tolist()
print(missing_currencies)
df.drop(missing_currencies, level=0, inplace=True)

In [ ]:
df.to_csv('data.csv')